In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math

from sklearn.tree            import DecisionTreeRegressor
from sklearn.neural_network  import MLPRegressor
from sklearn.linear_model    import LinearRegression
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics         import mean_squared_error
from sklearn.metrics         import r2_score
from sklearn.ensemble        import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.metrics         import mean_squared_error
from sklearn.preprocessing   import LabelEncoder,OneHotEncoder

In [45]:
train_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\Train.csv")

In [46]:
test_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\Test.csv")

In [47]:
train_df.isnull().sum()

id                                   0
name                                12
host_id                              0
host_name                           13
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       6982
reviews_per_month                 6982
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [48]:
def RemovalOfNulls(df):
    df = df.drop(['name','host_name'],axis=1)
    df['last_review'] = pd.to_datetime(df['last_review'])
    min_val = np.min(df['reviews_per_month'])
    min_row = df[df['reviews_per_month']==min_val].min()
    df['last_review'] = df['last_review'].fillna(min_row['last_review'])
    df['reviews_per_month'] = df['reviews_per_month'].fillna(min_row['reviews_per_month'])
    return df

In [49]:
train_df = RemovalOfNulls(train_df)

In [50]:
train_df.isnull().sum()

id                                0
host_id                           0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [51]:
train_df['log_(price+1)'] = np.log1p(train_df['price'])


In [52]:
train_df = train_df.drop(['price'],axis=1)

In [53]:
train_df =train_df.drop(['id','host_id'],axis=1)

In [54]:
train_df =train_df.drop(['neighbourhood'],axis=1)

In [55]:
train_df.columns

Index(['neighbourhood_group', 'latitude', 'longitude', 'room_type',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'log_(price+1)'],
      dtype='object')

In [41]:
train_df.dtypes

neighbourhood_group                       object
latitude                                 float64
longitude                                float64
room_type                                 object
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
log_(price+1)                            float64
dtype: object

In [56]:
train_df = train_df.drop(['last_review'],axis=1)

In [43]:
le = LabelEncoder()                                            # Fit label encoder
le.fit(train_df['neighbourhood_group'])
train_df['neighbourhood_group']=le.transform(train_df['neighbourhood_group'])    # Transform labels to normalized encoding.

le = LabelEncoder()
le.fit(train_df['neighbourhood'])
train_df['neighbourhood']=le.transform(train_df['neighbourhood'])

le = LabelEncoder()
le.fit(train_df['room_type'])
train_df['room_type']=le.transform(train_df['room_type'])

KeyError: 'neighbourhood'

In [57]:
train_df_with_dummies = pd.get_dummies(train_df)

In [58]:
train_df_with_dummies = train_df_with_dummies.drop(['last_review'],axis=1)

KeyError: "['last_review'] not found in axis"

In [59]:
train_df_with_dummies.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'log_(price+1)', 'neighbourhood_group_Bronx',
       'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan',
       'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island',
       'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [21]:
train_df.to_csv('train_LE_processed.csv', header=True, index=False) 

In [60]:
X=train_df_with_dummies.drop(['log_(price+1)'],axis=1)
y=train_df_with_dummies['log_(price+1)']

In [61]:
#y= target
#x = nums
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=24)

In [62]:
from sklearn.linear_model import LinearRegression

In [31]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_train = lr.predict(X_train)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train, lr.predict(X_train))))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, lr.predict(X_test))))


Train RMSE: 0.4989193281313136
Test RMSE: 0.5043963781347076


In [63]:
#'''Gradient Boosted Regressor'''
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.01)
GBoost.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [64]:
predicts2 = GBoost.predict(X_test)
predicts2

array([3.89224504, 5.59271609, 4.72931106, ..., 5.28558508, 4.37318577,
       5.41009997])

In [65]:
rmse_gb = math.sqrt(mean_squared_error(y_test,predicts2))
rmse_gb

0.4594547101963562

In [66]:
rmse_gb = math.sqrt(mean_squared_error(y_train,predicts2))
rmse_gb

ValueError: Found input variables with inconsistent numbers of samples: [23958, 10268]

In [67]:
np.sqrt(mean_squared_error(y_train,np.exp(GBoost.predict(X_train))-1)), np.sqrt(mean_squared_error(y_test,np.exp(GBoost.predict(X_test))-1))

(144.47115178368742, 143.0585741013181)

## TEST DATA

In [70]:
test_id_col = test_df['id']

In [71]:
test_df.isnull().sum()

id                                   0
name                                 4
host_id                              0
host_name                            8
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       3070
reviews_per_month                 3070
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [72]:
test_df = RemovalOfNulls(test_df)

In [73]:
test_df.isnull().sum()

id                                0
host_id                           0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [74]:
test_df = test_df.drop(['last_review'],axis=1)

In [75]:
test_df =test_df.drop(['id','host_id','neighbourhood'],axis=1)

In [76]:
test_df_with_dummies = pd.get_dummies(test_df)

In [77]:
test_df_with_dummies.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'neighbourhood_group_Bronx',
       'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan',
       'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island',
       'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [57]:
test_df_with_dummies['price'] = GBoost.predict(test_df_with_dummies)

In [78]:
test_df_with_dummies.to_csv('test_processed.csv', header=True, index=False) 

In [58]:
test_df_with_dummies['price'] = np.exp(test_df_with_dummies['price'])-1

In [59]:
submissions_GB = pd.concat([test_id_col, test_df_with_dummies['price']], axis=1) 


In [61]:
submissions_GB.to_csv('submission_GB_with_cat.csv', index=False) 

In [62]:
submissions_GB

,id,price
0,1525602,138.199755
1,30430185,171.003051
2,21354525,79.094231
3,35995074,74.346827
4,34392081,116.043088
...,...,...
14664,27730191,123.944463
14665,29676417,133.385710
14666,1813829,72.174097
14667,25017403,218.690913


In [63]:
submissions_GB.isnull().sum()

id       0
price    0
dtype: int64